In [ ]:
import sqlite3
import hashlib
import os

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('users.db')
cursor = conn.cursor()

# Create a table for users if it doesn't exist
cursor.execute('''
CREATE TABLE IF NOT EXISTS users (
    id INTEGER PRIMARY KEY,
    username TEXT NOT NULL UNIQUE,
    password TEXT NOT NULL
)
''')

# Function to hash a password
def hash_password(password):
    salt = os.urandom(16)
    hashed_password = hashlib.pbkdf2_hmac('sha256', password.encode('utf-8'), salt, 100000)
    return salt + hashed_password

# Function to verify a password
def verify_password(stored_password, provided_password):
    salt = stored_password[:16]
    stored_hash = stored_password[16:]
    provided_hash = hashlib.pbkdf2_hmac('sha256', provided_password.encode('utf-8'), salt, 100000)
    return stored_hash == provided_hash

# Function to register a new user
def register_user(username, password):
    hashed_password = hash_password(password)
    try:
        cursor.execute('INSERT INTO users (username, password) VALUES (?, ?)', (username, hashed_password))
        conn.commit()
        print(f"User {username} registered successfully!")
    except sqlite3.IntegrityError:
        print(f"User {username} already exists.")

# Function to authenticate a user
def login_user(username, password):
    cursor.execute('SELECT password FROM users WHERE username=?', (username,))
    user = cursor.fetchone()
    if user and verify_password(user[0], password):
        print(f"Login successful! Welcome, {username}.")
    else:
        print("Invalid username or password.")

# Example usage
register_user('user1', 'password1')
login_user('user1', 'password1')

# Close the database connection
conn.close()
